In [2]:
import os
import sys
import json

sys.path.append(os.path.abspath(os.pardir))

import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
%matplotlib inline

from tdparse.data_types import TargetCollection
from tdparse.helper import read_config, full_path

In [3]:
def extract_columns(model_result_fp, columns):
    all_dfs = []
    datasets = None
    for model, result_fp in model_result_fp.items():
        results_dataframe = pd.read_csv(result_fp, sep='\t')
        if datasets is None:
            datasets = results_dataframe['dataset']
        results_dataframe = results_dataframe.set_index('dataset')
        results_dataframe = results_dataframe[columns]
        new_col_names = {column : '{} {}'.format(model, column) for column in columns}
        results_dataframe = results_dataframe.rename(columns=new_col_names)

        all_dfs.append(results_dataframe)
    return pd.concat(all_dfs, axis=1)

def columns_avliable(model_result_fp):
    for model, result_fp in model_result_fp.items():
        results_dataframe = pd.read_csv(result_fp, sep='\t')
        return [col for col in results_dataframe.columns if 'Unnamed' not in col]
    

def add_mean(results_df, rounding=2):
    mean_results = results_df.mean()
    mean_results.name = 'Mean'
    results_df = results_df.append(mean_results).round(rounding)
    return results_df
    

# Mass Evaluation Result analysis

Below we are matching the name of the model to the result file for that model

In [10]:
root_folder = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
root_folder_join = lambda paths: os.path.join(root_folder, 'results', *paths)

target_dep_results_fp = root_folder_join(['Target Dependent Models', 'Target Dependent.tsv'])
target_dep_plus_results_fp = root_folder_join(['Target Dependent Models', 'Target Dependent+.tsv'])
tdparse_results_fp = root_folder_join(['TDParse Models','TDParse.tsv'])
tdparse_plus_results_fp = root_folder_join(['TDParse Models','TDParsePlus.tsv'])
lstm_results_fp = root_folder_join(['TDLstm', 'lstm', 'results file.tsv'])
tdlstm_results_fp = root_folder_join(['TDLstm', 'tdlstm', 'results file.tsv'])
tclstm_results_fp = root_folder_join(['TDLstm', 'tclstm', 'results file.tsv'])

model_result_fp = {'Target-Dep' : target_dep_results_fp, 'Target-Dep+' : target_dep_plus_results_fp,
                   'TDParse' : tdparse_results_fp, 'TDParse+' : tdparse_plus_results_fp,
                   'LSTM' : lstm_results_fp, 'TDLSTM' : tdlstm_results_fp, 'TCLSTM' : tclstm_results_fp}

Below we extract out the 3 way Macro F1 scores for each method on all of the dataset and combine them together into one table.

In [19]:
acc_f1_results = extract_columns(model_result_fp, ['3 Class Macro F1'])
acc_f1_results = add_mean(acc_f1_results)
acc_f1_results

,Target-Dep 3 Class Macro F1,Target-Dep+ 3 Class Macro F1,TDParse 3 Class Macro F1,TDParse+ 3 Class Macro F1,LSTM 3 Class Macro F1,TDLSTM 3 Class Macro F1,TCLSTM 3 Class Macro F1
Dong Twitter,65.60,66.20,65.90,67.2,61.70,59.70,59.10
Election Twitter,45.60,46.10,44.90,45.0,39.80,43.10,41.40
Mitchel,40.80,42.60,49.40,50.0,40.00,41.50,40.70
SemEval 14 Laptop,57.30,63.80,59.80,64.5,43.70,55.80,47.50
SemEval 14 Restaurant,59.90,62.60,61.20,63.0,45.70,56.70,48.00
YouTuBean,53.10,55.10,73.80,69.1,43.70,43.90,40.10
Mean,53.72,56.07,59.17,59.8,45.77,50.12,46.13


We do the same as above but for the Accuracy

In [14]:
acc_f1_results = extract_columns(model_result_fp, ['Accuracy'])
acc_f1_results = add_mean(acc_f1_results)
acc_f1_results

,Target-Dep Accuracy,Target-Dep+ Accuracy,TDParse Accuracy,TDParse+ Accuracy,LSTM Accuracy,TDLSTM Accuracy,TCLSTM Accuracy
Dong Twitter,68.40,68.20,68.80,69.80,64.70,63.60,63.90
Election Twitter,57.90,56.60,57.80,56.30,53.00,55.20,55.80
Mitchel,72.60,72.70,74.00,74.40,70.90,71.00,71.30
SemEval 14 Laptop,66.30,71.00,68.50,71.20,61.30,63.80,57.40
SemEval 14 Restaurant,74.80,76.30,75.70,76.70,71.80,72.90,68.20
YouTuBean,70.80,72.10,82.10,80.80,71.20,71.70,67.50
Mean,68.47,69.48,71.15,71.53,65.48,66.37,64.02


We can then look at the 2 way Macro F1 score which evaluates how well the models can predict Positive or Negative sentiment while still taking into account prediciting the neutral class

In [15]:
acc_f1_results = extract_columns(model_result_fp, ['2 Class F1'])
acc_f1_results = add_mean(acc_f1_results)
acc_f1_results

,Target-Dep 2 Class F1,Target-Dep+ 2 Class F1,TDParse 2 Class F1,TDParse+ 2 Class F1,LSTM 2 Class F1,TDLSTM 2 Class F1,TCLSTM 2 Class F1
Dong Twitter,61.00,62.6,61.10,62.90,57.2,53.70,52.20
Election Twitter,40.10,41.2,39.00,39.70,36.2,36.40,35.20
Mitchel,19.20,22.2,32.00,32.70,18.4,21.10,19.40
SemEval 14 Laptop,67.90,72.0,70.10,72.30,65.6,64.60,56.00
SemEval 14 Restaurant,72.40,74.7,73.20,75.60,68.5,67.20,58.00
YouTuBean,39.00,41.7,66.90,60.10,24.1,24.30,20.80
Mean,49.93,52.4,57.05,57.22,45.0,44.55,40.27


Now we look at the results of trying to predict the sentiment of all targets that are only in sentence that contain one distinct sentiment. This should be a result that is good for the LSTM model as it does not take into account any target information.

In [16]:
one_dist_sent = extract_columns(model_result_fp, ['3 Class Macro F1 for text with 1 distinct sentiments'])
one_dist_sent = add_mean(one_dist_sent)
one_dist_sent

,Target-Dep 3 Class Macro F1 for text with 1 distinct sentiments,Target-Dep+ 3 Class Macro F1 for text with 1 distinct sentiments,TDParse 3 Class Macro F1 for text with 1 distinct sentiments,TDParse+ 3 Class Macro F1 for text with 1 distinct sentiments,LSTM 3 Class Macro F1 for text with 1 distinct sentiments,TDLSTM 3 Class Macro F1 for text with 1 distinct sentiments,TCLSTM 3 Class Macro F1 for text with 1 distinct sentiments
Dong Twitter,65.60,66.20,65.90,67.2,61.70,59.70,59.10
Election Twitter,50.30,50.70,47.30,48.4,48.10,46.40,40.90
Mitchel,41.50,42.90,50.60,51.2,41.00,41.80,41.80
SemEval 14 Laptop,59.50,67.20,61.60,67.0,45.30,57.50,47.20
SemEval 14 Restaurant,61.10,65.80,60.40,65.1,47.10,56.40,45.50
YouTuBean,54.20,56.50,75.60,70.7,44.50,44.60,40.80
Mean,55.37,58.22,60.23,61.6,47.95,51.07,45.88


We do the same as above but for 2 distinct sentiments per sentence. We remove the Dong twitter dataset from this experiment due to it only containing Tweets with one distinct sentiment.

In [17]:
two_dist_sent = extract_columns(model_result_fp, ['3 Class Macro F1 for text with 2 distinct sentiments'])
two_dist_sent = two_dist_sent.drop('Dong Twitter')
two_dist_sent = add_mean(two_dist_sent)
two_dist_sent

,Target-Dep 3 Class Macro F1 for text with 2 distinct sentiments,Target-Dep+ 3 Class Macro F1 for text with 2 distinct sentiments,TDParse 3 Class Macro F1 for text with 2 distinct sentiments,TDParse+ 3 Class Macro F1 for text with 2 distinct sentiments,LSTM 3 Class Macro F1 for text with 2 distinct sentiments,TDLSTM 3 Class Macro F1 for text with 2 distinct sentiments,TCLSTM 3 Class Macro F1 for text with 2 distinct sentiments
Election Twitter,42.20,42.6,42.50,42.00,33.50,39.90,41.20
Mitchel,31.60,34.8,37.00,37.90,27.40,35.60,29.90
SemEval 14 Laptop,45.90,47.4,48.40,50.80,32.60,45.80,43.30
SemEval 14 Restaurant,53.10,52.6,57.30,55.30,38.30,51.90,48.00
YouTuBean,31.50,31.1,42.20,42.20,20.50,20.50,20.50
Mean,40.86,41.7,45.48,45.64,30.46,38.74,36.58


Lastly the same as above but for 3 distinct sentiments. As the majority of the datasets have very few 3 distinct sentiments per sentence we only look at the Twitter Elections datases (8.78% of the dataset contains 3 distinct sentiments)

In [18]:
three_dist_sent = extract_columns(model_result_fp, ['3 Class Macro F1 for text with 3 distinct sentiments'])
three_dist_sent = pd.DataFrame(three_dist_sent.loc['Election Twitter'])
three_dist_sent.sort_values('Election Twitter', ascending=False)

,Election Twitter
TDParse 3 Class Macro F1 for text with 3 distinct sentiments,44.8
TDLSTM 3 Class Macro F1 for text with 3 distinct sentiments,44.5
TDParse+ 3 Class Macro F1 for text with 3 distinct sentiments,44.0
TCLSTM 3 Class Macro F1 for text with 3 distinct sentiments,43.1
Target-Dep+ 3 Class Macro F1 for text with 3 distinct sentiments,42.4
Target-Dep 3 Class Macro F1 for text with 3 distinct sentiments,41.4
LSTM 3 Class Macro F1 for text with 3 distinct sentiments,30.2
